# Flux sampling

## Basic usage

The easiest way to get started with flux sampling is using the `sample` function in the `flux_analysis` submodule. `sample` takes at least two arguments: a cobra model and the number of samples you want to generate.

In [1]:
from cobra.test import create_test_model
from cobra.flux_analysis import sample

model = create_test_model("textbook")
s = sample(model, 100)
print(s)

[[ -8.14976854e-01  -3.27821650e-02  -1.46574322e-01 ...,   2.68156610e+00
    2.67747510e+00   6.47240918e+00]
 [ -1.08861837e-01  -3.42125437e-02  -2.18716843e+00 ...,   1.82347675e+00
    1.80282627e+00   7.76697573e+00]
 [ -2.02675537e+00  -1.98501949e-03  -1.08134678e+00 ...,   7.01126815e-01
    6.87172792e-01   8.39468212e+00]
 ..., 
 [ -1.70663570e+00  -3.65778562e-01  -2.99760586e-01 ...,   2.68523793e+00
    2.68502744e+00   7.25185024e+00]
 [ -3.71806904e-01  -1.91442667e-01  -5.48946125e-01 ...,   5.23255385e+00
    5.22722056e+00   3.34948182e+00]
 [ -4.69957186e+00  -3.33264065e+00  -1.08869832e+00 ...,   1.03239876e+00
    1.02537743e+00   8.60909963e+00]]


By default sample uses the `optgp` method based on the [method presented here](http://dx.doi.org/10.1371/journal.pone.0086587) as it is suited for larger models and can run in parallel. By default the sampler uses a single process. This can be changed by using the `processes` argument.

In [2]:
print("One process:")
%time s = sample(model, 1000)
print("Two processes:")
%time s = sample(model, 1000, processes=2)

One process:
CPU times: user 5.24 s, sys: 480 ms, total: 5.72 s
Wall time: 5.19 s
Two processes:
CPU times: user 84 ms, sys: 144 ms, total: 228 ms
Wall time: 2.66 s


Alternatively you can also user Artificial Centering Hit-and-Run for sampling by setting the method to `arch`. `arch` does not support parallel execution but has good convergence and is almost markovian.

In [3]:
s = sample(model, 100, method="arch")

In general setting up the sampler is expensive since initial search directions are generated by solving many linear programming problems. Thus, we recommend to generate as many samples as possible in one go. However, this might require finer control over the sampling procedure as described in the following section.

## Advanced usage

### Sampler objects

The sampling process can be controlled on a lower level by using the sampler classes directly.

In [4]:
from cobra.flux_analysis.sampling import OptGPSampler, ARCHSampler

Both sampler classes have standardized interfaces and take some additional argument. For instance the `thinning` factor. "Thinning" means only recording samples every n iterations. A higher thinning factors mean less correlated samples but also larger computation times. By default the samplers use a thinning factor of 100 which creates roughly uncorrelated samples. If you want less samples but better mixing feel free to increase this parameter. If you want to study convergence for your own model you might want to set it to 1 to obtain all iterates.

In [5]:
arch = ARCHSampler(model, thinning=10)

`OptGPSampler` has an additional `processes` argument specifying how many processes are used to create parallel sampling chains. This should be in the order of your CPU cores for maximum efficiency. As noted before class initialization can take up to a few minutes due to generation of initial search directions. Sampling on the other hand is quick.

In [6]:
optgp = OptGPSampler(model, processes=4)

### Sampling and validation

Both samplers have a sample function that generates samples from the intialized object and act like the `sample` function described above, only that this time it will oncly accept a single argument, the number of samples. For `OptGPSampler` the number of samples should be a mutiple of the number of processes, otherwise it will be increased to the nearest multiple automatically.

In [7]:
s1 = arch.sample(100)

s2 = optgp.sample(100)

You can call `sample` repeatedly and both samplers are optimized to generate large amount of samples without falling into "numerical traps". All sampler objects have a `validate` function in order to check if a set of points are feasible and give detailed information about feasibility violations in a form of a short code denoting feasibility. Here the short code is a combination of any of the following letters:

- "v" - valid point
- "l" - lower bound violation
- "u" - upper bound violation
- "e" - equality violation (meaning the point is not a steady state)

For instance for a random flux distribution (should not be feasible):

In [8]:
import numpy as np

bad = np.random.uniform(-1000, 1000, size=len(model.reactions))
arch.validate(np.atleast_2d(bad))

array(['le'], 
      dtype='<U3')

And for our generated samples:

In [9]:
arch.validate(s1)

array(['v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v',
       'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v', 'v'], 
      dtype='<U3')

### Batch sampling

Sampler objects are made for generating billions of samples, however using the `sample` function might quickly fill up your RAM when working with genome-scale models. Here, the `batch` method of the sampler objects might come in handy. `batch` takes two arguments, the number of samples in each badge and the number of batches. This will make for sense with a small example. 

Let's assume we want to qunatify what proportion of our samples will grow. For that we might want to generate 10 batches of 50 samples each and measure what percentage of the individual 100 samples show a growth rate larger than 0.1. Finally, we want to calculate the mean and standard deviation of those individual percentages.

In [10]:
# Biomass reaction is reaction number 12
print(model.reactions[12])

Biomass_Ecoli_core


In [14]:
counts = [np.mean(s[:, 12] > 0.1) for s in optgp.batch(100, 10)]
print("Usually {:.2f}% +- {:.2f}% grow...".format(np.mean(counts) * 100.0, np.std(counts) * 100.0))

Usually 0.40% +- 1.20% grow...
